In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

In [2]:
#reading in the csv file
staten_island_df = pd.read_csv("Resources/staten_island.csv")
staten_island_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   OBJECTID     390 non-null    int64  
 1   Borough      390 non-null    object 
 2   Offense      366 non-null    object 
 3   Latitude     390 non-null    float64
 4   Longitude    390 non-null    float64
 5   Public Area  24 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 18.4+ KB


In [3]:
#dividing the df into crimes and public institutions
r_crimes = staten_island_df.iloc[0:365, :]
r_crimes = r_crimes.drop(columns= ["Borough", "Public Area"])

In [4]:
r_crimes.head()

,OBJECTID,Offense,Latitude,Longitude
0,1,ROBBERY,40.643566,-74.074001
1,2,ROBBERY,40.635961,-74.161365
2,3,ROBBERY,40.643569,-74.098338
3,4,ROBBERY,40.630408,-74.142977
4,5,ROBBERY,40.630539,-74.076641


In [7]:
#same method for public institutions and reordering columns
r_public = staten_island_df.iloc[366:389, :]
r_public = r_public.drop(columns= ["Offense"])
r_public = r_public[["OBJECTID", "Public Area", "Latitude", "Longitude"]]

In [8]:
r_public.head()

,OBJECTID,Public Area,Latitude,Longitude
366,367,PARKS,40.614533,-74.074185
367,368,RECREATION AND WATERFRONT SITES,40.510392,-74.230405
368,369,PUBLIC LIBRARIES,40.595802,-74.063027
369,370,PUBLIC LIBRARIES,40.590395,-74.101138
370,371,PARKS,40.638748,-74.083899


In [12]:
#each location a crime took place

r_crimes_gdf = gpd.GeoDataFrame(
    r_crimes,
    geometry=gpd.points_from_xy(
        r_crimes["Longitude"],
        r_crimes["Latitude"],
    ),
    crs="EPSG:4326",
)

# each public institution

r_public_gdf = gpd.GeoDataFrame(
    r_public, 
    geometry=gpd.points_from_xy(
        r_public["Longitude"],
        r_public["Latitude"],
    ),
    crs="EPSG:4326",
)

# EPSG:3857 converts it to meters

r_crimes_gdf_proj = r_crimes_gdf.to_crs(crs="EPSG:3857")
r_public_gdf_proj = r_public_gdf.to_crs(crs="EPSG:3857")

# 1 miles would be 1609 meters

x = r_public_gdf_proj.buffer(1609).unary_union

crimes_per = r_crimes_gdf_proj["geometry"].intersection(x)

# print all the nearby points
test = (r_crimes_gdf[~crimes_per.is_empty])
test

,OBJECTID,Offense,Latitude,Longitude,geometry
0,1,ROBBERY,40.643566,-74.074001,POINT (-74.07400 40.64357)
1,2,ROBBERY,40.635961,-74.161365,POINT (-74.16137 40.63596)
4,5,ROBBERY,40.630539,-74.076641,POINT (-74.07664 40.63054)
5,6,ROBBERY,40.637135,-74.123473,POINT (-74.12347 40.63714)
6,7,ROBBERY,40.643996,-74.076502,POINT (-74.07650 40.64400)
...,...,...,...,...,...
359,360,ROBBERY,40.644719,-74.081327,POINT (-74.08133 40.64472)
360,361,ROBBERY,40.635738,-74.132124,POINT (-74.13212 40.63574)
362,363,ROBBERY,40.632740,-74.152082,POINT (-74.15208 40.63274)
363,364,ROBBERY,40.614616,-74.084716,POINT (-74.08472 40.61462)
